In [ ]:
# 필요 패키지 설치
!pip install -q transformers datasets peft trl bitsandbytes accelerate

## 1. 환경 설정

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 라이브러리 import
import os
import torch
from pathlib import Path
from typing import Dict, Any
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)
from trl import SFTTrainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. 설정

In [ ]:
# ===== 설정 =====
# 경로 설정
DATA_PATH = "/content/drive/MyDrive/lora_data/monster_style.jsonl"  # 학습 데이터 경로
OUTPUT_DIR = "/content/drive/MyDrive/lora_adapters/exaone-7.8b-monster-lora"  # 어댑터 저장 경로

# 모델 설정
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

# LoRA 설정
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# 학습 설정
NUM_EPOCHS = 3
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 8
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 512

# 출력 디렉토리 생성
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Data path: {DATA_PATH}")
print(f"Output dir: {OUTPUT_DIR}")

## 3. 양자화 및 LoRA 설정

In [ ]:
# 양자화 설정 (4-bit NF4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
print("Quantization config ready (4-bit NF4)")

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=TARGET_MODULES,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
print(f"LoRA config ready (r={LORA_R}, alpha={LORA_ALPHA})")

## 4. 모델 및 토크나이저 로드

In [ ]:
# 토크나이저 로드
print(f"Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
)

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("Tokenizer loaded")

In [ ]:
# 모델 로드 (4bit 양자화)
print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

# Gradient checkpointing을 위한 준비
model = prepare_model_for_kbit_training(model)
print("Model loaded")

In [ ]:
# LoRA 적용
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 5. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
dataset = load_dataset("json", data_files=DATA_PATH, split="train")
print(f"Loaded {len(dataset)} training samples")

# 샘플 확인
print("\n=== Sample Data ===")
print(dataset[0])

In [ ]:
# 데이터 포맷 함수 (EXAONE instruction format)
def format_instruction(sample: Dict[str, str]) -> str:
    """
    학습 데이터를 EXAONE instruction format으로 변환

    EXAONE format:
    [|system|]시스템 메시지[|endofturn|]
    [|user|]사용자 입력[|endofturn|]
    [|assistant|]어시스턴트 응답[|endofturn|]
    """
    system_msg = "당신은 몬스터입니다. 붕괴된 문법, 반복, 의성어를 사용하여 말하세요."
    user_msg = sample["input"]

    formatted = (
        f"[|system|]{system_msg}[|endofturn|]\n"
        f"[|user|]{user_msg}[|endofturn|]\n"
        f"[|assistant|]{sample['output']}[|endofturn|]"
    )

    return formatted

# 포맷 테스트
print("\n=== Formatted Sample ===")
print(format_instruction(dataset[0]))

## 6. 학습 설정

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    group_by_length=True,
    report_to="none",
    remove_unused_columns=False,
)

print("Training arguments configured")

In [ ]:
# SFTTrainer 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=format_instruction,
    max_seq_length=MAX_SEQ_LENGTH,
)

print("Trainer ready")

## 7. 학습 실행

In [ ]:
print("=" * 60)
print("Monster Style LoRA Training for EXAONE 7.8B")
print("=" * 60)
print("\n목적: 몬스터 말투(문장 구조, 반복, 붕괴된 문법, 의성어/의태어, 광기 표현)만 학습")
print("주의: 게임 로직, humanity 변수, 상태 전이, semantic role은 포함하지 않음\n")
print("Starting training...")
print("=" * 60 + "\n")

# 학습 시작
trainer.train()

## 8. 모델 저장

In [ ]:
# LoRA 어댑터 저장
print(f"Saving model to: {OUTPUT_DIR}")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("\n" + "=" * 60)
print("Training completed!")
print("=" * 60)

## 9. 추론 테스트

In [ ]:
# 추론 테스트 함수
def test_inference(prompt: str) -> str:
    """학습된 LoRA로 추론 테스트"""
    system_msg = "당신은 몬스터입니다. 붕괴된 문법, 반복, 의성어를 사용하여 말하세요."
    formatted_prompt = (
        f"[|system|]{system_msg}[|endofturn|]\n"
        f"[|user|]{prompt}[|endofturn|]\n"
        f"[|assistant|]"
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response.split("[|assistant|]")[-1].replace("[|endofturn|]", "").strip()

    return response

In [ ]:
# 테스트 실행
test_prompts = [
    "안녕하세요, 만나서 반갑습니다.",
    "몬스터처럼 말해줘.",
    "다음 문장을 몬스터처럼 말해줘.\n\n배가 고파요.",
]

print("=== Inference Test ===")
for prompt in test_prompts:
    response = test_inference(prompt)
    print(f"\nInput: {prompt}")
    print(f"Monster Response: {response}")
    print("-" * 50)

## 10. 저장된 어댑터 로드 (별도 세션에서 사용시)

In [ ]:
# 이 셀은 별도 세션에서 학습된 어댑터를 로드할 때 사용
# 주석 해제 후 실행

'''
from peft import PeftModel

ADAPTER_PATH = "/content/drive/MyDrive/lora_adapters/exaone-7.8b-monster-lora"
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

# 기본 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# LoRA 어댑터 로드
model = PeftModel.from_pretrained(model, ADAPTER_PATH)
model.eval()

print("Adapter loaded successfully!")
'''